# PBSA-CUDA — computation of solvation energy
This notebook will walk us through computing the solvation energy of a Conformer using the Poisson-Boltzmann Surface Area (PBSA) method.

This notebook will demonstrate how to calculate for a single ligand, but this example can be easily extended across the entire thermodynamic energy cycle, including the calculation of the protein, the ligand, and the resultant complex.

## 0) Setup
See [Quickstart](../index.ipynb#imports) for more details on the setup.

## 0.0) Imports

In [1]:
import os
import json
from pathlib import Path

import requests
import tengu

## 0.1) Credentials

In [2]:
TOKEN = os.getenv("TENGU_TOKEN")
# You might have a custom deployment url, by default it will use https://tengu.qdx.ai
TENGU_URL = os.getenv("TENGU_URL") or 'https://tengu.qdx.ai'

## 0.2) Configuration
Let's set some global variables that define our project.

In [3]:
DESCRIPTION = "pbsa-solvation-calculation-notebook"
TAGS = ['tengu-py', 'pbsa', 'convert']
WORK_DIR = Path.home() / "qdx" / "pbsa-cuda-tengu-py-demo"

## 0.3) Build your tengu client


In [4]:
#|hide
if WORK_DIR.exists():
    client = tengu.Provider(workspace=WORK_DIR,access_token=TOKEN, url=TENGU_URL)
    await client.nuke()

In [5]:
os.makedirs(WORK_DIR, exist_ok=True)

client = await tengu.build_provider_with_functions(
    access_token=TOKEN, url=TENGU_URL, workspace=WORK_DIR, batch_tags=TAGS
)

## 0.4) Download Asipirin SDF from PubChem

In [6]:
# Convert aspirin to a QDXF file so we can use it for this demo
SMILES_STRING = "CC(=O)OC1=CC=CC=C1C(=O)O"
SDF_LINK = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/smiles/{SMILES_STRING}/record/SDF?record_type=3d"

file_path = f'{WORK_DIR}/aspirin.sdf'
with open(file_path, 'wb') as f:
    f.write(requests.get(SDF_LINK).content)

## 0.5) Convert SDF file to QDXF format
QDXF is the central molecule format of the Tengu API, so before we use the Dubai module (as a pre-requisite to running PBSA) we must convert our SDF file to QDXF.

In [7]:
# We need to specify storage > file size to ensure that we have allocated enough resources for the convert module
(ligand,) = await client.convert("SDF", Path(file_path), resources={"storage": 5000})
await ligand.get()

ligand = await ligand.get()
ligand = ligand[0]

Exception: (<ModuleFailureReason.RUN: 'RUN'>, ModuleInstanceCommonFailureContext(stdout='', stderr='Error: SDFParseError { line_number: 57, state: DataItems, ty: ParseDataKey }', syserr='/home/ryan/tengu/jobs/,18d2ede3a1e.2336901'))

In [ ]:
LIGAND_FILEPATH = Path(f'{WORK_DIR}/aspirin_pbsa.qdxf.json')
json.dump(ligand, open(LIGAND_FILEPATH, 'w'))

## 1.1) Run Dubai to infer partial charges
We need to run Dubai to infer partial charges, as PBSA-CUDA requires access to the partial charges.

In [ ]:
DUBAI_RESOURCES = {
    "gpus": 1,
    "storage": 1024_000,
    "walltime": 60,
}

(ligand_partial_charges,) = await client.dubai(LIGAND_FILEPATH, resources=DUBAI_RESOURCES)
ligand_partial_charges = await ligand_partial_charges.get()

os.remove(LIGAND_FILEPATH)
json.dump(ligand_partial_charges, open(LIGAND_FILEPATH, 'w'))

self.typeinfo={'k': 'object', 't': 'Conformer'} self.provider=<tengu.provider.Provider object at 0x119d3e6d0> self.id=UUID('3f1ddbcb-07f4-450a-ad87-aaff874d9b2b')


## 1.0) Set PBSA specific module configuration
In this stage, we set configuration for the PBSA module.

In [ ]:
PBSA_RESOURCES = {
    "gpus": 1,
    "storage": 1024_000,
    "walltime": 60,
}

PBSA_PARAMETERS = {
    'solute_dielectric': 1.0,
    'solvent_dielectric': 78.54,
    'solvent_radius': 0.14,
    'ion_concentration': 0.0,
    'temperature': 298.0,
    'spacing': 0.04,
    'sasa_gamma': 2.26778,
    'sasa_beta': 3.84928,
    'sasa_n_samples': 1000,
    'convergence': 0.00001,
    'box_size_factor': 2.0
}

## 1.2) Run PBSA-CUDA
Finally, we run PBSA-CUDA to compute solvation energy of our ligand. 

In [ ]:
solvation_energy = await client.pbsa(LIGAND_FILEPATH, PBSA_PARAMETERS, resources=PBSA_RESOURCES)

solvation_energy = await solvation_energy[0].get()
polar_solvation_energy = solvation_energy[1].get()
non_polar_solvation_energy = solvation_energy[2].get()

-0.038586
